# Research Outputs Selection

## 1. Project Setup

In [4]:
# Libraries
import json

### 1.1 Project Configurations

In [1]:
# Project Config
proj_json_file = r"C:\Users\zooka\git_repos\learning\python\jinja_and_json\202008.03_001_project.json"
proj_dir = r"C:\Users\zooka\git_repos\learning\python\jinja_and_json"

### 1.2 Load Configurations

In [7]:
# Opening JSON file
f = open(proj_json_file)

# Returns JSON object as a dictionary
proj_json = json.load(f)

# Closing file
f.close()

## Project Header

In [11]:
project = proj_json['project']

print('------------------------')
print('Project ID : ' + project["id"])
print(project["proj_no"])
print(project["task_no"])
print(project["proj_title"])
print(project["princ_invest"])
print('------------------------')

------------------------
Project ID : p2
202008.03
001
Ovarian Cancer
Prof Penelope Webb
------------------------
